### QUESTION 10: 

For each time period described in Question 6, perform the same grid search above for GradientBoostingRegressor (with corresponding time window length). Does the cross-validation test error change? Are the best parameter set you find in each period agree with those you found above?

In [1]:
hash_tags = ['#gohawks','#gopatriots','#nfl','#patriots','#sb49','#superbowl']

In [2]:
import pickle

def save_object(data, fileName):
    with open('pynb_data/'+fileName + ".pickle", 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        
def load_object(fileName):
    try:
        with open('pynb_data/'+fileName + ".pickle", 'rb') as f:
            data = pickle.load(f)
            return data
    except IOError:
        print("Could not read file: " + fileName)

In [3]:
import json

def getMinAndMaxTs(tag):
    filename = 'data/tweets_'+tag+'.txt'
    max_ts = 0
    min_ts = 1552522378
    with open(filename) as f:
        for line in f:
            json_object = json.loads(line)
            timestamp = json_object['citation_date']
            if(timestamp < min_ts):                
                min_ts = timestamp
            
            if(timestamp > max_ts):
                max_ts = timestamp
                
    return [min_ts,max_ts]

tagsToMinTs = {}
tagsToMaxTs = {}
for tag in hash_tags:
    ts_list = getMinAndMaxTs(tag)
    tagsToMinTs[tag] = (ts_list[0])
    tagsToMaxTs[tag] = (ts_list[1])    

In [4]:
import math
import datetime
import pytz


def getLocalHour(timestamp):
    d = datetime.datetime.fromtimestamp(timestamp)
    pst = pytz.timezone('America/Los_Angeles')
    d = pst.localize(d)
    return d.hour

def getWindowNumber(start_ts, curr_ts, window):
    elapsed = (curr_ts - start_ts)/(window*1.0)
    windowNum = math.ceil(elapsed)
    return windowNum    

def getFeatures(start_ts,end_ts,window):
    windowToTweets = {}
    windowToRetweets = {}
    windowToFollowerCount = {}
    windowToMaxFollowers = {}
    features = []
    labels = []
    
    for tag in hash_tags:
        filename = 'data/tweets_'+tag+'.txt'
        with open(filename) as f:
            for line in f:
                json_object = json.loads(line)
                timestamp = json_object['citation_date']
            
                if timestamp < start_ts or timestamp > end_ts:                            
                    continue
                
                key = getWindowNumber(start_ts,timestamp,window)
    #             print(key)
                if key not in windowToTweets.keys():
                    windowToTweets[key]=0
                windowToTweets[key]+=1
            
                retweetCount = json_object['metrics']['citations']['total']        
            
                if key not in windowToRetweets.keys():
                    windowToRetweets[key]=0
                windowToRetweets[key]+=retweetCount
        
                followerCount = json_object['author']['followers']
                if key not in windowToFollowerCount.keys():
                    windowToFollowerCount[key]=0
                windowToFollowerCount[key]+=followerCount
        
                if key not in windowToMaxFollowers.keys():
                    windowToMaxFollowers[key]=0
                windowToMaxFollowers[key] = max(windowToMaxFollowers[key],followerCount)            
            
    for period in range(start_ts,end_ts,window):
        key = getWindowNumber(start_ts,period,window)
        tweetCount = windowToTweets.get(key, 0)
        retweetCount = windowToRetweets.get(key,0)
        followerCount = windowToFollowerCount.get(key,0)
        maxFollowers = windowToMaxFollowers.get(key,0)

        h = getLocalHour(key)
            
        feature = [tweetCount, retweetCount, followerCount, maxFollowers, h]
        features.append(feature)
                
        nextKey = getWindowNumber(start_ts, period + window, window)
        labels.append(windowToTweets.get(nextKey,0))
                
    return features,labels

In [6]:
# http://aplunket.com/random-forest-regressor/
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

#tp1
tp1_window_size = 3600 # 1 hour window size
tp1_start_ts = tp1_window_size * math.floor(tagsToMinTs[tag]/(tp1_window_size*1.0))
tp1_end_ts = 1422806400
# features,labels = getFeatures(tp1_start_ts,tp1_end_ts,tp1_window_size)
print(len(features))
print(labels)
# save_object(features, "q7_tp1_features_{}".format(tag))
# save_object(labels, "q7_tp1_labels_{}".format(tag))

features = load_object("q7_tp1_features")
labels = load_object("q7_tp1_labels")
    


param_grid = {
  'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
  'max_features': ['auto', 'sqrt'],
  'min_samples_leaf': [1, 2, 4],
  'min_samples_split': [2, 5, 10],
  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

model = GradientBoostingRegressor(random_state=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')
grid.fit(features, labels)

print(grid.best_score_)
print(grid.best_params_)

440
[111, 89, 110, 100, 137, 169, 215, 353, 569, 533, 530, 544, 525, 628, 611, 675, 260, 256, 233, 342, 402, 334, 258, 119, 88, 275, 155, 173, 336, 160, 291, 479, 632, 563, 304, 528, 651, 534, 626, 614, 376, 398, 496, 387, 243, 258, 203, 65, 145, 34, 20, 42, 180, 115, 185, 687, 885, 954, 704, 981, 1032, 825, 884, 799, 474, 799, 724, 473, 459, 337, 334, 236, 125, 103, 82, 133, 218, 93, 192, 309, 500, 612, 430, 710, 783, 687, 686, 670, 428, 712, 697, 614, 619, 542, 582, 417, 434, 347, 139, 21, 312, 544, 1141, 1839, 2459, 3087, 2493, 6380, 10260, 6702, 7271, 39421, 14572, 8757, 22584, 18564, 4389, 2640, 1997, 1413, 914, 709, 721, 666, 1510, 1123, 1475, 2183, 1964, 1975, 1158, 1718, 1546, 1413, 1080, 1221, 1293, 1270, 7962, 1361, 654, 658, 443, 409, 287, 204, 219, 221, 336, 282, 434, 690, 840, 957, 511, 775, 46, 85, 705, 786, 524, 817, 811, 670, 3310, 2254, 995, 609, 329, 261, 343, 667, 975, 739, 838, 928, 870, 858, 844, 843, 842, 767, 783, 1087, 1101, 1263, 931, 966, 814, 793, 416, 409, 2

In [7]:
# http://aplunket.com/random-forest-regressor/
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

#tp1
tp1_window_size = 3600 # 1 hour window size
tp1_start_ts = tp1_window_size * math.floor(tagsToMinTs[tag]/(tp1_window_size*1.0))
tp1_end_ts = 1422806400
# features,labels = getFeatures(tp1_start_ts,tp1_end_ts,tp1_window_size)
print(len(features))
print(labels)
# save_object(features, "q7_tp1_features_{}".format(tag))
# save_object(labels, "q7_tp1_labels_{}".format(tag))

features = load_object("q7_tp2_features")
labels = load_object("q7_tp2_labels")
    


param_grid = {
  'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
  'max_features': ['auto', 'sqrt'],
  'min_samples_leaf': [1, 2, 4],
  'min_samples_split': [2, 5, 10],
  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

model = GradientBoostingRegressor(random_state=0)
tp2grid = GridSearchCV(estimator=model, param_grid=param_grid, cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')
tp2grid.fit(features, labels)

print(tp2grid.best_score_)
print(tp2grid.best_params_)

440
[111, 89, 110, 100, 137, 169, 215, 353, 569, 533, 530, 544, 525, 628, 611, 675, 260, 256, 233, 342, 402, 334, 258, 119, 88, 275, 155, 173, 336, 160, 291, 479, 632, 563, 304, 528, 651, 534, 626, 614, 376, 398, 496, 387, 243, 258, 203, 65, 145, 34, 20, 42, 180, 115, 185, 687, 885, 954, 704, 981, 1032, 825, 884, 799, 474, 799, 724, 473, 459, 337, 334, 236, 125, 103, 82, 133, 218, 93, 192, 309, 500, 612, 430, 710, 783, 687, 686, 670, 428, 712, 697, 614, 619, 542, 582, 417, 434, 347, 139, 21, 312, 544, 1141, 1839, 2459, 3087, 2493, 6380, 10260, 6702, 7271, 39421, 14572, 8757, 22584, 18564, 4389, 2640, 1997, 1413, 914, 709, 721, 666, 1510, 1123, 1475, 2183, 1964, 1975, 1158, 1718, 1546, 1413, 1080, 1221, 1293, 1270, 7962, 1361, 654, 658, 443, 409, 287, 204, 219, 221, 336, 282, 434, 690, 840, 957, 511, 775, 46, 85, 705, 786, 524, 817, 811, 670, 3310, 2254, 995, 609, 329, 261, 343, 667, 975, 739, 838, 928, 870, 858, 844, 843, 842, 767, 783, 1087, 1101, 1263, 931, 966, 814, 793, 416, 409, 2

/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [8]:
# http://aplunket.com/random-forest-regressor/
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

#tp1
tp1_window_size = 3600 # 1 hour window size
tp1_start_ts = tp1_window_size * math.floor(tagsToMinTs[tag]/(tp1_window_size*1.0))
tp1_end_ts = 1422806400
# features,labels = getFeatures(tp1_start_ts,tp1_end_ts,tp1_window_size)
print(len(features))
print(labels)
# save_object(features, "q7_tp1_features_{}".format(tag))
# save_object(labels, "q7_tp1_labels_{}".format(tag))

features = load_object("q7_tp3_features")
labels = load_object("q7_tp3_labels")
    


param_grid = {
  'max_depth': [10, 20, 40, 60, 80, 100, 200, None],
  'max_features': ['auto', 'sqrt'],
  'min_samples_leaf': [1, 2, 4],
  'min_samples_split': [2, 5, 10],
  'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}

model = GradientBoostingRegressor(random_state=0)
tp3grid = GridSearchCV(estimator=model, param_grid=param_grid, cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')
tp3grid.fit(features, labels)

print(tp3grid.best_score_)
print(tp3grid.best_params_)

144
[1300, 1343, 1226, 1302, 1224, 319, 245, 231, 230, 237, 284, 266, 470, 281, 332, 400, 1401, 1493, 1709, 1494, 1593, 1463, 1107, 1716, 1861, 1611, 1544, 1560, 1770, 1676, 1820, 1569, 1697, 8499, 9648, 9991, 5736, 9791, 14318, 15471, 15151, 15241, 13539, 13771, 13805, 15622, 14181, 14508, 13067, 13880, 12876, 12221, 11966, 12615, 11745, 11469, 11083, 11119, 10466, 11276, 10371, 11089, 10611, 10115, 9768, 9986, 9730, 8193, 7300, 7410, 11060, 9837, 9391, 9201, 9217, 8763, 9215, 8532, 9360, 10176, 10267, 10557, 10779, 12747, 20744, 19360, 27623, 25137, 34867, 32175, 34416, 23369, 22546, 20954, 27243, 25419, 23214, 24047, 36027, 25872, 23175, 26587, 30303, 28063, 22709, 26830, 23483, 27450, 34990, 25681, 35025, 45904, 45773, 37186, 23920, 23983, 19550, 17554, 24798, 22708, 13866, 14376, 12927, 12568, 14721, 19292, 17790, 14340, 12789, 28000, 16210, 21143, 35905, 41702, 24998, 17300, 13897, 12540, 9347, 8027, 6915, 5890, 3177, 610]
-2530331.111027386
{'max_depth': 20, 'max_features': 'aut